In [ ]:
# === PASO 1: Limpieza y combinación de bases ===
import pandas as pd

# Cargar archivos CSV
clientes = pd.read_csv('base_clientes_final.csv')
transacciones = pd.read_csv('base_transacciones_final.csv')

# Limpiar nombres de columnas por si acaso
clientes.rename(columns=lambda x: x.strip(), inplace=True)
transacciones.rename(columns=lambda x: x.strip(), inplace=True)

# Combinar las bases usando 'id',,,
base_completa = transacciones.merge(clientes, on='id', how='left')

# Llenar nulos en giro_comercio
base_completa['giro_comercio'].fillna("SIN CLASIFICAR", inplace=True)

# Vista previa
print("Vista previa de la base combinada:")
print(base_completa.head())

# Guardar base combinada
base_completa.to_csv('base_completa.csv', index=False)
print("\n¡Archivo guardado como 'base_completa.csv'!")


In [5]:
# PASO 2: Calcular meses_distintos por cliente–comercio
# Asegurarse que la fecha sea tipo datetime
base_completa['fecha'] = pd.to_datetime(base_completa['fecha'])

# Crear columna año-mes
base_completa['año_mes'] = base_completa['fecha'].dt.to_period('M')

# Agrupar por cliente y comercio para contar meses distintos
frecuencia_mensual = (
    base_completa.groupby(['id', 'comercio'])['año_mes']
    .nunique()
    .reset_index(name='meses_distintos')
)


In [ ]:
#CÓDIGO DEL PASO 3:

import matplotlib.pyplot as plt

# Histograma de meses distintos por cliente-comercio
frecuencia_mensual['meses_distintos'].hist(bins=12, edgecolor='black')
plt.xlabel("Meses distintos con gasto en el comercio")
plt.ylabel("Número de cliente-comercio")
plt.title("Distribución de meses distintos por cliente-comercio")
plt.xticks(range(1, 13))  # Meses del 1 al 12
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
# === CONTINUACIÓN DESDE PASO 2 ===

# 3. Calcular promedio, std y número de transacciones por cliente-comercio
agrupado_montos = base_completa.groupby(['id', 'comercio'])['monto'].agg(
    monto_prom='mean',
    monto_std='std',
    num_transacciones='count'
).reset_index()

# Eliminar columnas duplicadas antes de hacer el merge para evitar conflictos de sufijos
cols_to_drop = ['monto_prom', 'monto_std', 'num_transacciones']
frecuencia_mensual = frecuencia_mensual.drop(columns=[col for col in cols_to_drop if col in frecuencia_mensual.columns])

# Unir con frecuencia_mensual
frecuencia_mensual = frecuencia_mensual.merge(agrupado_montos, on=['id', 'comercio'], how='left')

# 4. Calcular rupturas de meses consecutivos
def contar_saltes(meses):
    meses_ordenados = sorted(meses.unique())
    diferencias = [meses_ordenados[i+1] - meses_ordenados[i] for i in range(len(meses_ordenados)-1)]
    return sum([d.n != 1 for d in diferencias])  # d.n convierte a valor numérico

rupturas = (
    base_completa.groupby(['id', 'comercio'])['año_mes']
    .apply(contar_saltes)
    .reset_index(name='meses_no_consecutivos')
)

# Unir con frecuencia_mensual
frecuencia_mensual = frecuencia_mensual.merge(rupturas, on=['id', 'comercio'], how='left')

# 5. Crear etiquetas

# General: recurrente si cumple ≥7 meses, estabilidad en monto y pocos saltos
frecuencia_mensual['es_recurrente'] = (
    (frecuencia_mensual['meses_distintos'] >= 7) &
    (frecuencia_mensual['monto_std'] < 100) &
    (frecuencia_mensual['meses_no_consecutivos'] <= 2)
).astype(int)

# Fuerte: exactamente 12 meses de gasto
frecuencia_mensual['es_recurrente_fuerte'] = (
    frecuencia_mensual['meses_distintos'] == 12
).astype(int)

# 6. Vista previa y exportación
print(frecuencia_mensual.sort_values(by='meses_distintos', ascending=False).head())

frecuencia_mensual.to_csv('frecuencia_mensual_etiquetada.csv', index=False)
print("\n¡Archivo guardado como 'frecuencia_mensual_etiquetada.csv'!")


In [18]:
# === PASO 6: Entrenamiento del modelo de clasificación ===
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# 1. Seleccionar las features (X) y la etiqueta (y)
features = ['meses_distintos', 'monto_prom', 'monto_std', 'num_transacciones', 'meses_no_consecutivos']
X = frecuencia_mensual[features]
y = frecuencia_mensual['es_recurrente']

# 2. Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. Crear y entrenar el modelo
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 4. Hacer predicciones
y_pred = clf.predict(X_test)

# 5. Evaluar el modelo
print("\n=== Reporte de clasificación ===")
print(classification_report(y_test, y_pred))

print("\n=== Matriz de confusión ===")
print(confusion_matrix(y_test, y_pred))



=== Reporte de clasificación ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4736
           1       1.00      1.00      1.00      1954

    accuracy                           1.00      6690
   macro avg       1.00      1.00      1.00      6690
weighted avg       1.00      1.00      1.00      6690


=== Matriz de confusión ===
[[4736    0]
 [   0 1954]]
